In [34]:
import json
import pennylane as qml
import pennylane.numpy as np
import scipy

def abs_dist(rho, sigma):
    """A function to compute the absolute value |rho - sigma|."""
    polar = scipy.linalg.polar(rho - sigma)
    return polar[1]

def word_dist(word):
    """A function which counts the non-identity operators in a Pauli word"""
    return sum(word[i] != "I" for i in range(len(word)))


# Produce the Pauli density for a given Pauli word and apply noise

def noisy_Pauli_density(word, lmbda):
    """
       A subcircuit which prepares a density matrix (I + P)/2**n for a given Pauli
       word P, and applies depolarizing noise to each qubit. Nothing is returned.

    Args:
            word (str): A Pauli word represented as a string with characters I,  X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.
    """


    # Put your code here #
    n = len(word)
    I = np.identity(2**n)
    
    def circuit():
        for i in range(n):
            if word[i] == 'I':
                qml.Identity(i)
            if word[i] == 'X':
                qml.PauliX(i)
            if word[i] == 'Y':
                qml.PauliY(i)
            if word[i] == 'Z':
                qml.PauliZ(i)
    P = qml.matrix(circuit)()
                          
    rho = (I+P)/2**n
    qml.QubitDensityMatrix(rho, wires=range(n))
    
    for i in range(n):        
        qml.DepolarizingChannel(lmbda, wires=i)

# Compute the trace distance from a noisy Pauli density to the maximally mixed density

def maxmix_trace_dist(word, lmbda):
    """
       A function compute the trace distance between a noisy density matrix, specified
       by a Pauli word, and the maximally mixed matrix.

    Args:
            word (str): A Pauli word represented as a string with characters I, X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.

    Returns:
            float: The trace distance between two matrices encoding Pauli words.
    """


    # Put your code here #
    n = len(word)
    dev = qml.device("default.mixed", wires = n)
    
    @qml.qnode(dev)
    def circuit():
        noisy_Pauli_density(word, lmbda)
        return qml.density_matrix(wires = list(range(n)))
    
    
    rho = circuit()
    
    rho_0 = np.identity(2**n)/2**n
    
    return 0.5*abs_dist(rho,rho_0).trace()


def bound_verifier(word, lmbda):
    """
       A simple check function which verifies the trace distance from a noisy Pauli density
       to the maximally mixed matrix is bounded by (1 - lambda)^|P|.

    Args:
            word (str): A Pauli word represented as a string with characters I, X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.

    Returns:
            float: The difference between (1 - lambda)^|P| and T(rho_P(lambda), rho_0).
    """


    # Put your code here #
    P = word_dist(word)
    trace_dis = maxmix_trace_dist(word, lmbda)
    return (1 - lmbda)**P - trace_dis




In [35]:
bound_verifier("XXI", 0.7)

(0.0877777777777777+0j)